In [1]:
import os
from bs4 import BeautifulSoup as BS
import csv
import pandas as pd
from time import time 

In [2]:
path = './prova/'

In [3]:
def intro_plot(soup,index=0):
    try:
        Node = soup.find_all('p')[index]
        lst = [Node.text]
        nextNode = Node
        while True:
            nextNode = nextNode.next_sibling
            try:
                tag_name = nextNode.name
            except AttributeError:
                tag_name = ""
            if tag_name == "p":
                lst.append(nextNode.text)
            else:
                break
    except:
        lst = None
    return lst

In [4]:
def soup_parser(soup,info=None,next_tag='td',class_name=None):
    if next_tag:
        element = soup.find('th', text=info)
        if not element:
            element = 'NA'
        else:
            if info=='Release date':
                element = element.find_next(next_tag).text.split()[0]
            else:
                element = element.find_next(next_tag).text
    else:
        element = soup.find('h1').text
    return element

In [5]:
if not os.path.exists('tsv_files'):
    os.makedirs('tsv_files')

In [6]:
columns = ['title', 'intro', 'plot', 'film_name', 'director', 'producer', 'writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget']
exceptions = []
start = time()
for subdir, dirs,files in os.walk(path):
    for file in files:
        try:
            soup = BS(open(path+file, encoding="utf8"), "html.parser")
            
            title = soup_parser(soup=soup,next_tag=None,class_name='firstHeading')
            intro = intro_plot(soup=soup)
            plot = intro_plot(soup=soup,index=len(intro))
            director = soup_parser(soup=soup, info='Directed by')
            producer = soup_parser(soup=soup, info='Produced by')
            writer = soup_parser(soup=soup, info='Written by')
            starred = soup_parser(soup=soup, info='Starring')
            music = soup_parser(soup=soup, info='Music by')
            date = soup_parser(soup=soup,info='Release date')
            duration = soup_parser(soup=soup, info='Running time')
            country = soup_parser(soup=soup, info='Country')
            language = soup_parser(soup=soup, info='Language')
            budget = soup_parser(soup=soup, info='Budget')
            
            all_info = [title,intro,plot,title,director,producer,writer,starred,music,date,duration,country,language,budget]
            with open('tsv_files/%s.tsv' %(file[:-5]), 'wt',encoding="utf-8") as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(columns)
                tsv_writer.writerow(all_info)

        except:
            print('Could not parse html file:', file)
            exceptions.append(file)
print("Execution time is:",time()-start)

Execution time is: 64.00750684738159


In [9]:
pd.read_csv("tsv_files/article-800.tsv", sep="\t")

,title,intro,plot,film_name,director,producer,writer,starring,music,release date,runtime,country,language,budget
0,Hardboiled Rose,['Hardboiled Rose is a 1929 American part-talk...,"[""A Southern belle (Loy) must work in a gambli...",Hardboiled Rose,F. Harmon Weight,NaN,Darryl F. Zanuck (story)Robert Lord (screenpla...,"Myrna LoyWilliam Collier, Jr.John Miljan",NaN,March,5 reels (silent version)6 reels (sound version),United States,English,NaN
